<a href="https://colab.research.google.com/github/Gokul0Krishna/exp/blob/main/exp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip "/content/archive.zip"

Archive:  /content/archive.zip
  inflating: human detection dataset/0/0.png  
  inflating: human detection dataset/0/1.png  
  inflating: human detection dataset/0/10.png  
  inflating: human detection dataset/0/100.png  
  inflating: human detection dataset/0/101.png  
  inflating: human detection dataset/0/102.png  
  inflating: human detection dataset/0/103.png  
  inflating: human detection dataset/0/104.png  
  inflating: human detection dataset/0/105.png  
  inflating: human detection dataset/0/106.png  
  inflating: human detection dataset/0/107.png  
  inflating: human detection dataset/0/108.png  
  inflating: human detection dataset/0/109.png  
  inflating: human detection dataset/0/11.png  
  inflating: human detection dataset/0/110.png  
  inflating: human detection dataset/0/111.png  
  inflating: human detection dataset/0/112.png  
  inflating: human detection dataset/0/113.png  
  inflating: human detection dataset/0/114.png  
  inflating: human detection dataset/0/115.p

In [ ]:
import os
image=[]
lable=[]
l=["No people","people"]
for i in os.listdir("/content/human detection dataset/"):
  for j in os.listdir("/content/human detection dataset/"+i):
    image.append("/content/human detection dataset/"+i+"/"+j)
    lable.append(l[int(i)])

In [ ]:
iamge=[]
import cv2
import numpy as np
from PIL import Image
for i in image:
    img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28,28))
    img = np.array(img, dtype=np.float32) / 255.0
    img = img.flatten()
    iamge.append(img)

In [ ]:
import pandas as pd
ndf=pd.DataFrame({
    "image":iamge,
    "label":lable
})
df=pd.get_dummies(ndf,columns=['label'],dtype=float)
df.columns = df.columns.str.replace("label_", "")
df.head()

,image,No people,people
0,"[0.3529412, 0.43529412, 0.5568628, 0.5568628, ...",0.0,1.0
1,"[0.3137255, 0.14509805, 0.3764706, 0.53333336,...",0.0,1.0
2,"[0.34117648, 0.2627451, 0.13333334, 0.19215687...",0.0,1.0
3,"[0.6156863, 0.654902, 0.67058825, 0.6745098, 0...",0.0,1.0
4,"[0.6627451, 0.70980394, 0.74509805, 0.22352941...",0.0,1.0


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['No people']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'No people'}, axis=1)
              .sort_values('No people', ascending=True))
  xs = counted['No people']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('No people', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('No people')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['people']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'people'}, axis=1)
              .sort_values('people', ascending=True))
  xs = counted['people']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('people', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('people')
_ = plt.ylabel('count()')

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest,ytrain,ytest = train_test_split(df['image'],df[l],random_state=42, test_size=0.2)
xtest, xval,ytest,yval = train_test_split(xtest,ytest,random_state=42, test_size=0.5)

In [ ]:
import torch
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
import torch.nn as nn

In [ ]:
from PIL import Image
import numpy as np
class customdataset(Dataset):
    def __init__(self,x,y,transform=None):
        self.x=x
        self.y=y
        self.transform=transform

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, index):
        imgpath=self.x.iloc[index]
        if self.transform:
            image=Image.open(imgpath).convert("RGB")
            image=self.transform(image)
        label_row = self.y.iloc[index, 0:2]
        label_tensor = torch.tensor(label_row.values.astype(np.float32))
        label_index = torch.argmax(label_tensor).item()

        return image,label_index

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((640, 640)),  # YOLO typically uses square inputs
    transforms.ToTensor(),
])

In [ ]:
traindataset=customdataset(x=xtrain,y=ytrain,transform=preprocess)
testdataset=customdataset(x=xtest,y=ytest,transform=preprocess)
valdataset=customdataset(x=xval,y=yval,transform=preprocess)

In [ ]:
trainloader=DataLoader(traindataset,batch_size=64,shuffle=True)
testloader=DataLoader(testdataset,batch_size=64,shuffle=True)
valloader=DataLoader(valdataset,batch_size=64,shuffle=True)

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
import torch
import ultralytics
ultralytics.checks()
# Load model (ensure yolov5 repo is in your path)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.eval()

Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (AMD EPYC 7B12)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.2/107.7 GB disk)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-7-27 Python-3.11.13 torch-2.6.0+cu124 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 125MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [ ]:
import torch.optim as optim
criterion = nn.BCELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

NameError: name 'os' is not defined